# Multi-Image Transformation

This example notebook demonstrates how to load and transform a series of RHEED images taken during sequential azimuthal rotations. 

The rotation starts at $\alpha = 0^\circ$, where the electron beam is aligned along the $[11\bar{2}]$ direction, and ends at $\alpha = 30^\circ$, corresponding to alignment along the $[1\bar{1}0]$ direction.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import xrheed
from xrheed.plotting.overview import plot_images
from xrheed.preparation.filters import high_pass_filter
from xrheed.preparation.alignment import find_incident_angle

## Image Preprocessing and Correction

To begin, we load a series of images and apply the necessary corrections.  
Due to minor manipulator inaccuracies, each image may exhibit slight variations in positional shifts along the $S_x$ and $S_y$ axes.

Typically, these variations change linearly during the rotation.  

For instance, the shift along $S_y$ could be prepared as a linear space between the values recorded for the first and last images.  

The same approach applies to the azimuthal angle, α.

In [ ]:
image_dir = Path("example_data")
image_paths = list(sorted(image_dir.glob("Si_111_r3Ag_thA_phi*.raw")))

n_images = len(image_paths)

# Define alphas for each image (include manipulator inaccuracies)
alpha_coords = np.linspace(0.0, 29.6, n_images)


# Load data into a stack by providing alpha coordinate
arheed = xrheed.load_data(
    image_paths, 
    plugin="dsnp_arpes_raw", 
    stack_dim="alpha", 
    stack_coords=alpha_coords
)

# Define center position for each image
center_x = -5.8
center_y_vect = np.linspace(0.0, 3.5, n_images) + 3.0

# Set the center using single value or a vector
arheed.ri.set_center_manual(center_x=center_x, center_y=center_y_vect)

# Set screen ROI
arheed.ri.screen_roi_width = 60
arheed.ri.screen_roi_height = 60

arheed.ri.incident_angle = find_incident_angle(arheed.isel(alpha=0))

# Optionally use high pass filter (could be applied to the whole stack)
arheed = high_pass_filter(arheed, sigma=2.0, threshold=0.7)

# Finally plot loaded stack 
plot_images(arheed, ncols=2, show_specular_spot=True, auto_levels=0.2)
plt.show()

## Prepare a Lattice Object for the Expected Reconstruction

To verify the alignment, we create a lattice object representing the expected (√3 × √3) R30° reconstruction.  

This lattice will later be used by the `Ewald` object for spot calculation and overlay.


In [ ]:
from xrheed.kinematics.lattice import Lattice

# Si(111)-(1x1)
si_111_1x1 = Lattice.from_surface_hex(a=3.84, label="(1x1)")

# Si(111)-(r3xr3)R30
si_111_r3 = Lattice.from_surface_hex(a=3.84 * np.sqrt(3), label="(√3 x √3)")
si_111_r3.rotate(30)

## Generate Ewald Object and Overlay Expected Spot Positions

We generate an `Ewald` objects and use it to overlay the expected spot positions on each image.

This allows for visual verification of the alignment and consistency across the dataset.

> **Note**: The `Ewald` object could take a stack of images. In such case a `stack_index` property could be used to select a particular image from a stack.


In [ ]:
from xrheed.kinematics.ewald import Ewald

ew = Ewald(si_111_r3, arheed)

fig, axs = plt.subplots(n_images, 1, sharex=True, figsize=(4.0, 14))

for idx in range(n_images):
    ax = axs[idx]
    # Select the image from the stack
    ew.stack_index = idx

    ew.plot(
        ax=ax,
        show_image=True,
        vmin=3,
        vmax=35,
        marker="o",
        facecolor="none",
        edgecolor="c",
        s=50,
    )
    if idx < n_images - 1:
        ax.set_xlabel("")

plt.tight_layout()
plt.show()

## Transform Image Stack to kx–ky Space

If the alignment appears satisfactory, we proceed by transforming the whole image stack into the kx–ky space.  

This step uses the arguments `rotate=True` and `point_symmetry=True`.  

- The `rotate=True` option applies a rotation around the image center by the azimuthal angle α.  
- The `point_symmetry=True` option adds a mirrored image rotated by 180°, which is valid for most crystallographic structures.


In [ ]:
from xrheed.conversion.image import transform_image_to_kxky

trans_arheed = transform_image_to_kxky(arheed, point_symmetry=True, rotate=True)

## Compute the Mean Along the Azimuthal Dimension

Calculate the mean of the dataset along the azimuthal (α) dimension.  
This operation reduces the data to a single representative image, averaging over all angular orientations.


In [ ]:
trans_arheed_mean = trans_arheed.mean(dim="alpha")

## Prepare the Final Plot

Finally, we generate an image that displays the mean intensity across all frames, along with overlays of the (1×1) and (√3 × √3) R30° reciprocal lattices for comparison.

As observed, the diffraction spots are elongated along several directions.  
This elongation is a characteristic feature of RHEED, resulting from the finite domain size, which affects the shape of the diffraction spots.

Nonetheless, in this representation, it is straightforward to identify specific surface reconstructions, their symmetries, and mutual orientations.


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

trans_arheed_mean.plot(ax=ax, cmap="gray", add_colorbar=False, vmin=5, vmax=17)

si_111_r3.plot_reciprocal(ax=ax, facecolor="none", edgecolor="c", s=150)
si_111_1x1.plot_reciprocal(ax=ax, facecolor="none", edgecolor="m", s=50)

ax.set_aspect(1)
ax.set_facecolor("black")
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
ax.legend(loc="upper right")

plt.show()